In [463]:
import configparser
import requests
from psycopg2 import connect
from psycopg2.extras import execute_values
CONFIG = configparser.ConfigParser()
CONFIG.read(r'C:\Users\nchan6\Documents\db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [699]:
def get_url(mapserver, id):
    "This prints the base_url for the request"
    url = "https://insideto-gis.toronto.ca/arcgis/rest/services/{}/MapServer/{}/query".format(mapserver, id)
    return(url)

In [704]:
def get_objectid(where_clause):
    "This gets the objectid of all available fields"
    base_url = get_url("cot_geospatial2", 2)
    query = {"where":"{}".format(where_clause), "returnIdsOnly": "true", "f": "json"}
    r = requests.get(base_url, params = query, verify=False)
    objectid_json = r.json()
    return  objectid_json['objectIds'] ;

In [707]:
def get_layer(mapserver, id, output_table):
    
    """
    This function calls to the GCCview rest API and inserts the outputs to the output table in the postgres database.
    
    Parameters
    ----------
    mapserver : string
        The name of the mapserver that host the desire layer
        
    id : int
        The id of desire layer
        
    output_table: string
        Name of table that returned row will be inserted into
    """
    base_url = get_url({},{}).format(mapserver, id)
    print('Getting object_id...')
    object_id = get_objectid('OBJECTID>=1')
    print('object_id returned')
    
    # first call to api, check if the object_ids are too long
    query = {"where":"OBJECTID in {}".format(tuple(object_id)),
    "text": "",
    "objectIds":"", 
    "outFields": "*",
    "outSR": '4326',         
    "returnGeometry": "true",
    "returnTrueCurves": "false",
    "returnIdsOnly": "false",
    "returnCountOnly": "false",
    "returnZ": "false",
    "returnM": "false",
    "returnDistinctValues": "false",
    "returnExtentsOnly": "false",
    "f": "json"}

    
    r = requests.get(base_url, params = query, verify=False)
    
    # if the url is too long    
    if str(r) == '<Response [414]>':
        print('url too long, proceed with for loop')
        loop_id = []
        for i in range(0, len(object_id), 49):    
            if i == 0:
                stuff = (object_id[0:i+50])
                loop_id.append(stuff)
            else:
                stuff = (object_id[i+1:i+50])
                loop_id.append(stuff)
       
        # loop though 50 of objectid each time
        for i in loop_id:
            query = {"where":"OBJECTID in {}".format(tuple(i)),
                    "text": "",
                    "objectIds":"", 
                    "outFields": "*",
                    "outSR": '4326',         
                    "returnGeometry": "true",
                    "returnTrueCurves": "false",
                    "returnIdsOnly": "false",
                    "returnCountOnly": "false",
                    "returnZ": "false",
                    "returnM": "false",
                    "returnDistinctValues": "false",
                    "returnExtentsOnly": "false",
                    "f": "json"}
            r = requests.get(base_url, params = query, verify=False)
            return_json = r.json()
            features = return_json['features']
            rows = []
            for feature in features:
                row = (feature['attributes']['CENTRELINE_ID'], 
                       feature['attributes']['TRANS_ID_CREATE'],
                       feature['attributes']['OBJECTID'],
                       feature['attributes']['MI_PRINX'],
                       feature['attributes']['REC_ID'],
                       feature['attributes']['TYPE_CODE'],
                       feature['attributes']['TYPE_DESC'],
                       feature['attributes']['STATUS_CODE'],
                       feature['attributes']['STATUS_DESC'],
                       'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['paths'][0]) +')')

                rows.append(row)

            # send to the database
            sql='INSERT INTO {} (geo_id, trans_id_create, object_id, mi_prinx, rec_id, type_code, type_desc, status_code, status_desc, geometry) VALUES %s'.format(output_table)

            with con:
                with con.cursor() as cur:
                    execute_values(cur, sql, rows) 

    # if url is not too long 
    else: 
        return_json = r.json()
        features = return_json['features']
        rows = []
        for feature in features:
            row = (feature['attributes']['CENTRELINE_ID'], 
                   feature['attributes']['TRANS_ID_CREATE'],
                   feature['attributes']['OBJECTID'],
                   feature['attributes']['MI_PRINX'],
                   feature['attributes']['REC_ID'],
                   feature['attributes']['TYPE_CODE'],
                   feature['attributes']['TYPE_DESC'],
                   feature['attributes']['STATUS_CODE'],
                   feature['attributes']['STATUS_DESC'],
                   'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['paths'][0]) +')')
            
            rows.append(row)
            
        # send to database    
        sql='INSERT INTO {} (geo_id, trans_id_create, object_id, mi_prinx, rec_id, type_code, type_desc, status_code, status_desc, geometry) VALUES %s'.format(output_table)
        
        with con:
            with con.cursor() as cur:
                execute_values(cur, sql, rows) 


In [709]:
get_layer('cot_geospatial2', 2, 'natalie.gcc_table')

Getting object_id...
object_id returned


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


url too long, proceed with for loop


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTT